# X-Health 

Importando as bibliotecas e os pacotes necessários:

In [20]:
# Manipulação de arquivos e dados 
from pathlib import Path 
import pandas as pd
import numpy as np 

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

Para esse exercício, foi utilizado o dataset **"dataset_2021-5-26-10-14.csv"** presente na pasta **data** do repositório X-health. 

Carregando os dados: 

In [29]:
# identificando diretório base 
base_dir = Path.cwd()

# caminho para a pasta contendo o dataset  
data_dir = base_dir.parent / "data"

# caminho para o dataset 
df_path = data_dir / "dataset_2021-5-26-10-14.csv"

# importando o arquivo csv para um dataframe pandas
df = pd.read_csv(df_path, sep="\t", encoding="utf-8")

Visualização parcial do dataset: 

In [28]:
# visualizando as primeiras e últimas linhas do dataframe
df.head()
df.tail() 

,default_3months,ioi_36months,ioi_3months,valor_por_vencer,valor_vencido,valor_quitado,quant_protestos,valor_protestos,quant_acao_judicial,acao_judicial_valor,...,dividas_vencidas_qtd,falencia_concordata_qtd,tipo_sociedade,opcao_tributaria,atividade_principal,forma_pagamento,valor_total_pedido,month,year,default
117268,0,32.679779,16.582511,10470.620143,0.0,540673.324711,0,0.000000,0,0.0,...,0,0,sociedade empresaria limitada,simples nacional,com de telefones e equip p/ comunicacoes,30/60/90,13290.156794,1,2019,1
117269,0,41.894875,16.649383,3349.884555,0.0,3373.145777,0,0.000000,0,0.0,...,0,0,empresario (individual),simples nacional,comercio,missing,4574.831977,4,2018,1
117270,0,132.525884,18.188413,893.233266,0.0,3362.163427,0,0.000000,0,0.0,...,0,0,empresario-mei(microempreendedor individual),simples nacional,com de moveis e estofados,missing,944.324251,1,2017,1
117271,0,26.930798,16.337212,43969.125732,0.0,13889.897554,2,622.546407,0,0.0,...,0,0,sociedade empresaria limitada,simples nacional,servicos de telefonia fixa,30/60/90/120,5295.273974,11,2019,1
117272,1,11.334081,15.431495,19281.291534,0.0,40008.605168,0,0.000000,0,0.0,...,0,0,sociedade empresaria limitada,simples nacional,comercio,30/60/90,3584.906269,2,2018,1


## Análise Exploratória dos Dados 

### 1 - Entendendo a Estrutura do Dataset

 #### Variáveis e Entradas: 
  *  Verificando quantas entradas (linhas) e quantas variáveis (colunas) possui

In [22]:
print(f"O dataset contém {df.shape[0]} entradas e {df.shape[1]} variáveis")

O dataset contém 117273 entradas e 22 variáveis


*  Verificando os nomes das variáveis e seus tipos

In [23]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117273 entries, 0 to 117272
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   default_3months              117273 non-null  int64  
 1   ioi_36months                 117273 non-null  float64
 2   ioi_3months                  117273 non-null  float64
 3   valor_por_vencer             117273 non-null  float64
 4   valor_vencido                117273 non-null  float64
 5   valor_quitado                117273 non-null  float64
 6   quant_protestos              117273 non-null  int64  
 7   valor_protestos              117273 non-null  float64
 8   quant_acao_judicial          117273 non-null  int64  
 9   acao_judicial_valor          117273 non-null  float64
 10  participacao_falencia_valor  117273 non-null  float64
 11  dividas_vencidas_valor       117273 non-null  float64
 12  dividas_vencidas_qtd         117273 non-null  int64  
 13 

*  A descrição das variáveis:

    -  `default_3months` : Quantidade de default nos últimos 3 meses
    -  `ioi_36months`: Intervalo médio entre pedidos (em dias) nos últimos 36 meses
    -  `ioi_3months`: Intervalo médio entre pedidos (em dias) nos últimos 3 meses
    -  `valor_por_vencer`: Total em pagamentos a vencer do cliente B2B, em Reais
    -  `valor_vencido`: Total em pagamentos vencidos do cliente B2B, em Reais
    -  `valor_quitado`: Total (em Reais) pago no histórico de compras do cliente B2B 
    -  `quant_protestos` : Quantidade de protestos de títulos de pagamento apresentados no Serasa
    -  `valor_protestos` : Valor total (em Reais) dos protestos de títulos de pagamento apresentados no Serasa
    - `quant_acao_judicial` : Quantidade de ações judiciais apresentadas pelo Serasa
    -  `acao_judicial_valor` : Valor total das ações judiciais (Serasa) 
    -  `participacao_falencia_valor` : Valor total (em Reais) de falências apresentadas pelo Serasa
    -  `dividas_vencidas_valor` : Valor total de dívidas vencidas (Serasa)
    -  `dividas_vencidas_qtd` : Quantidade total de dívidas vencidas (Serasa)
    -  `falencia_concordata_qtd` : Quantidade de concordatas (Serasa)
    -  `tipo_sociedade` : Tipo de sociedade do cliente B2B 
    -  `opcao_tributaria` : Opção tributária do cliente B2B
    -  `atividade_principal` : Atividade principal do cliente B2B
    -  `forma_pagamento` : Forma de pagamento combinada para o pedido
    -  `valor_total_pedido` : Valor total (em Reais) do pedido em questão
    -  `month` : Mês do pedido
    -  `year` : Ano do pedido
    -  `default` : Status do pedido: default = 0 (pago em dia), default = 1 (pagamento não-realizado, calote concretizado)

    
    A coluna `default` é a variável alvo

*  Verificando se existem valores ausentes:

In [53]:
df.isnull().sum()

default_3months                0
ioi_36months                   0
ioi_3months                    0
valor_por_vencer               0
valor_vencido                  0
valor_quitado                  0
quant_protestos                0
valor_protestos                0
quant_acao_judicial            0
acao_judicial_valor            0
participacao_falencia_valor    0
dividas_vencidas_valor         0
dividas_vencidas_qtd           0
falencia_concordata_qtd        0
tipo_sociedade                 0
opcao_tributaria               0
atividade_principal            0
forma_pagamento                0
valor_total_pedido             0
month                          0
year                           0
default                        0
dtype: int64

In [52]:
# verificando valores ausentes 
df.isna().sum() #contagem de NaN

default_3months                0
ioi_36months                   0
ioi_3months                    0
valor_por_vencer               0
valor_vencido                  0
valor_quitado                  0
quant_protestos                0
valor_protestos                0
quant_acao_judicial            0
acao_judicial_valor            0
participacao_falencia_valor    0
dividas_vencidas_valor         0
dividas_vencidas_qtd           0
falencia_concordata_qtd        0
tipo_sociedade                 0
opcao_tributaria               0
atividade_principal            0
forma_pagamento                0
valor_total_pedido             0
month                          0
year                           0
default                        0
dtype: int64

In [24]:
# verificação de valores "missing" 
(df == "missing").sum() 

default_3months                    0
ioi_36months                       0
ioi_3months                        0
valor_por_vencer                   0
valor_vencido                      0
valor_quitado                      0
quant_protestos                    0
valor_protestos                    0
quant_acao_judicial                0
acao_judicial_valor                0
participacao_falencia_valor        0
dividas_vencidas_valor             0
dividas_vencidas_qtd               0
falencia_concordata_qtd            0
tipo_sociedade                   608
opcao_tributaria               18325
atividade_principal              592
forma_pagamento                32477
valor_total_pedido                 0
month                              0
year                               0
default                            0
dtype: int64

As colunas "tipo_sociedade", "opcao_tributaria", "atividade_principal" e "forma_pagamento" contém valores ausentes "missing". 

Entendendo as categorias existentes em cada coluna para decidir como lidar com os valores "missing": 

In [50]:
# Análise das categorias

# Lista de colunas categóricas
colunas_categoricas = ["tipo_sociedade", "opcao_tributaria", "atividade_principal", "forma_pagamento"]

# verificando a frequência absoluta e relativa para cada coluna
for coluna in colunas_categoricas:
    print(f"\ncoluna: {coluna}")

    df_freq = pd.DataFrame({
        "Freq absoluta": df[coluna].value_counts(),
        "Freq relativa (%)": df[coluna].value_counts(normalize=True) * 100
        })
    
    display(df_freq)

# verificando % de valores missing por coluna
print("\nPercentual de Valores Missing por Coluna:")
df_missing = pd.DataFrame((df[colunas_categoricas] == "missing").mean() * 100, columns=["% de Missing"]).round(2)
display(df_missing)


coluna: tipo_sociedade


,Freq absoluta,Freq relativa (%)
tipo_sociedade,,
sociedade empresaria limitada,60020,51.179726
empresario (individual),27055,23.070101
empresa individual respons limitada empresaria,19201,16.372908
empresario-mei(microempreendedor individual),6886,5.871769
sociedade anonima fechada,2482,2.116429
missing,608,0.518448
cooperativa,567,0.483487
sociedade simples limitada,121,0.103178
outras formas de associacao,106,0.090387



coluna: opcao_tributaria


,Freq absoluta,Freq relativa (%)
opcao_tributaria,,
simples nacional,87993,75.032616
missing,18325,15.625933
lucro real,7807,6.657116
lucro presumido,2757,2.350925
isento,391,0.333410



coluna: atividade_principal


,Freq absoluta,Freq relativa (%)
atividade_principal,,
com de equipamentos de informatica,22629,19.296002
com de telefones e equip p/ comunicacoes,9851,8.400058
papelaria,9075,7.738354
com de moveis e estofados,8226,7.014402
com de eletrodomesticos,8156,6.954713
...,...,...
sindicato de empregados,2,0.001705
bco mult priv nacional c/ partic estrangeira,2,0.001705
promocao de eventos,2,0.001705



coluna: forma_pagamento


,Freq absoluta,Freq relativa (%)
forma_pagamento,,
missing,32477,27.693501
30/60/90,29415,25.082500
30/60/90/120,7264,6.194094
28/42/56,6483,5.528127
28 dias,3375,2.877900
...,...,...
80/110/140/170/200/230,2,0.001705
110 dias,2,0.001705
36x (0+36),1,0.000853



Percentual de Valores Missing por Coluna:


,% de Missing
tipo_sociedade,0.52
opcao_tributaria,15.63
atividade_principal,0.50
forma_pagamento,27.69


### Análise de valores ausentes

Os resultados mostram que as colunas `tipo_sociedade` e `atividade_principal` apresentam apenas **~ 0.5%** de valores ausentes. 

Como o % é **baixo** (menos que 5%), a abordagem recomendada é remover essas linhas.

Por outro lado, as colunas `opcao_tributaria` **(15.63%)** e `forma_pagamento` **(27.69)** apresentam percentuais elevados de valores missing, que podem ou não ser aleatórios. 

Quando uma variável tem muitos valores ausentes, é importante investigar **se essas ausências seguem algum padrão**, pois isso pode impactar as decisões do modelo.

Para isso, foi verificado qual o **tipo de "missing data"** em `opcao_tributaria` e `forma_pagamento`:

*  **MCAR (Missing Completely at Random**): A ausência ocorre sem qualquer padrão e não está relacionada a nenhuma variável. 
*  **MAR (Missing at Random)**: A ausência não é totalmente aleatória, mas está relacionada a outras variáveis no dataset.
*  **MNAR (Missing Not at Random)**: A ausencia não é aleatória, está relacionada ao próprio valor da variável ausente. 

In [54]:
# Removendo linhas com "missing" em `tipo_sociedade` e `atividade_principal`

df = df.loc[(df["tipo_sociedade"] != "missing") & (df["atividade_principal"] != "missing")].copy()

# 